In [1]:
from z3 import *

In [3]:
# n-nuzzle, p-prittle, s-skipple, c-crottle, d-dupple
pallet, (n,p,s,c,d) = EnumSort('pallet', ['n', 'p','s','c','d'])

carry=IntSort()
number=IntSort()
#truck has a number, and a certain number of type of pallets it carries
truck=Function('truck',number,pallet,carry)

maxWeight,maxPallets=Ints('maxWeight maxPallets')

#number of each pallet
noN,noP,noS,noC,noD=Ints('noN noP noS noC noD')

#for iterating
i=Int('i')

#1-weight of all possible combined pallets per truck smaller than maxWeight
def checkTruckWeight():
    return(
        ForAll(i,
            truck(i,n)*800+truck(i,p)*1300+truck(i,s)*1000+truck(i,c)*1500+truck(i,d)*400<=maxWeight
         )
    )

#2-each pallet type must be greater than 0
def minPalletsPerTruck():
    return(
        ForAll(i,
            And(
                truck(i,n)>=0,
                truck(i,p)>=0,
                truck(i,s)>=0,
                truck(i,c)>=0,
                truck(i,d)>=0,                
            )
        )
    )

#3-total number of pallets in a truck <=8
def maxPalletsPerTruck():
    return(
        ForAll(i,
            truck(i,n)+truck(i,p)+truck(i,s)+truck(i,c)+truck(i,d)<=maxPallets
        )
    )

#4-max 2 trucks have skipples
def truckSkipple():
    return(
        And(
            truck(1,s)==0,
            truck(2,s)==0,
            truck(3,s)==0,
            truck(4,s)==0,
        )
    )

#5-max 1 nuzzle per truck
def truckNuzzle():
    return(
        ForAll(i,
            truck(i,n)<=1
        )
    )

# slv=Solver()
slv=Optimize()

#max weight,pallets
slv.add(maxWeight==8000)
slv.add(maxPallets==8)

#number of pallets per category
slv.add(noN==4) #total weight of nuzzle 4*800=3200
slv.add(noS==8) #total weight of skipple 8*1000=8000
slv.add(noC==8) #total weight of crottle 8*1500=12000
slv.add(noD==12)#total weight of dupple 12*400=4800
#for prittles we have unknown number but we know total max capacity of trucks to be 6*8000=48000
#48000-3200-8000-12000-4800= 20000 max weight for prittle
# 20000/1300 ~ max 15 skipples
slv.add(noP==15) 

#constraints from the methods above
slv.add(checkTruckWeight())
slv.add(minPalletsPerTruck())
slv.add(maxPalletsPerTruck())
slv.add(truckSkipple())
slv.add(truckNuzzle())

#each pallet from all possible trucks must sum up the the number of pallets
slv.add( truck(1,n)+truck(2,n)+truck(3,n)+truck(4,n)+truck(5,n)+truck(6,n)==noN )
slv.add( truck(1,p)+truck(2,p)+truck(3,p)+truck(4,p)+truck(5,p)+truck(6,p)==noP )
slv.add( truck(1,s)+truck(2,s)+truck(3,s)+truck(4,s)+truck(5,s)+truck(6,s)==noS )
slv.add( truck(1,c)+truck(2,c)+truck(3,c)+truck(4,c)+truck(5,c)+truck(6,c)==noC )
slv.add( truck(1,d)+truck(2,d)+truck(3,d)+truck(4,d)+truck(5,d)+truck(6,d)==noD )

print(slv.check())
# print(slv.model())
m = slv.model()
# print(m)

print("         ","nuzzles","prittles","skipples","crottles","dupples","weight")
for j in range(1,7):
    print(   
        'truck',j,"     ",
         m.evaluate(truck(j,n)),"     ",
         m.evaluate(truck(j,p)),"     ",
         m.evaluate(truck(j,s)),"     ",
         m.evaluate(truck(j,c)),"     ",
         m.evaluate(truck(j,d)),"     ",
         m.evaluate(truck(j,n)*800+truck(j,p)*1300+truck(j,s)*1000+truck(j,c)*1500+truck(j,d)*400)
    )

sat
          nuzzles prittles skipples crottles dupples weight
truck 1       1       0       0       4       3       8000
truck 2       0       4       0       1       3       7900
truck 3       0       4       0       1       3       7900
truck 4       1       4       0       1       1       7900
truck 5       1       0       7       0       0       7800
truck 6       1       3       1       1       2       8000
